In [0]:
import pandas as pd
import numpy as np
from google.colab import drive,files
from sklearn.preprocessing import OneHotEncoder

In [0]:
drive.mount('/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
DATA_PATH = '/gdrive/Shared drives/Predictive_Project/data/'
application_test_df = pd.read_csv(DATA_PATH + 'application_test.csv',index_col=0)
application_train_df = pd.read_csv(DATA_PATH + 'application_train.csv',index_col=0)
bureau_df = pd.read_csv(DATA_PATH + 'bureau.csv',index_col=1)
bureau_balance_df = pd.read_csv(DATA_PATH + 'bureau_balance.csv',index_col=0)
installments_payments_df = pd.read_csv(DATA_PATH + 'installments_payments.csv')
POS_CASH_balance_df = pd.read_csv(DATA_PATH + 'POS_CASH_balance.csv')
previous_application_df = pd.read_csv(DATA_PATH + 'previous_application.csv')
sample_submission_df = pd.read_csv(DATA_PATH + 'sample_submission.csv')
credit_balance_df = pd.read_csv(DATA_PATH + 'credit_card_balance.csv',index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
di = {"X":0,"C":0}

bureau_balance_df.replace({"STATUS": di})


bureau_balance_agg_dict = {"STATUS": ["max"]}
bureau_balance_agg = bureau_balance_df.groupby('SK_ID_BUREAU').agg({**bureau_balance_agg_dict})
bureau_balance_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in bureau_balance_agg.columns.tolist()])

In [0]:
# Bureau and bureau_balance numeric features
bureau_df = bureau_df.merge(bureau_balance_agg,on='SK_ID_BUREAU')
bureau_agg_dict = {
        'DAYS_CREDIT': ['max'],
        'CREDIT_DAY_OVERDUE': ['max'],
        'DAYS_CREDIT_ENDDATE': ['max'],
        'DAYS_ENDDATE_FACT':['max'],
        'AMT_CREDIT_MAX_OVERDUE': ['max'],
        'CNT_CREDIT_PROLONG' : ['max'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max'],
        'AMT_CREDIT_SUM_OVERDUE': ['max'],
        'AMT_CREDIT_SUM_LIMIT': ['max'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum']
    }

bureau_agg = bureau_df.groupby('SK_ID_CURR').agg({**bureau_agg_dict})
bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

In [0]:
bureau_agg.head()

,BURO_DAYS_CREDIT_MAX,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_ENDDATE_FACT_MAX,BURO_AMT_CREDIT_MAX_OVERDUE_MAX,BURO_CNT_CREDIT_PROLONG_SUM,BURO_AMT_CREDIT_SUM_MAX,BURO_AMT_CREDIT_SUM_MEAN,BURO_AMT_CREDIT_SUM_SUM,BURO_AMT_CREDIT_SUM_DEBT_MAX,BURO_AMT_CREDIT_SUM_OVERDUE_MAX,BURO_AMT_CREDIT_SUM_LIMIT_MAX,BURO_AMT_ANNUITY_MAX,BURO_AMT_ANNUITY_MEAN
SK_ID_CURR,,,,,,,,,,,,,,
100001,-49,0,1778.0,-544.0,NaN,0,378000.0,207623.571429,1453365.000,373239.0,0.0,0.000,10822.5,3545.357143
100002,-103,0,780.0,-36.0,5043.645,0,450000.0,108131.945625,865055.565,245781.0,0.0,31988.565,0.0,0.000000
100005,-62,0,1324.0,-123.0,0.000,0,568800.0,219042.000000,657126.000,543087.0,0.0,0.000,4261.5,1420.500000
100010,-1138,0,689.0,-1138.0,NaN,0,675000.0,495000.000000,990000.000,348007.5,0.0,0.000,NaN,NaN
100013,-1210,0,-567.0,-549.0,19305.000,0,1262250.0,518070.015000,2072280.060,0.0,0.0,NaN,0.0,0.000000


In [0]:
# installments
installments_payments_df['PAYMENT_MISS_FLAG'] = 0
installments_payments_df.loc[installments_payments_df['AMT_PAYMENT'] > installments_payments_df['AMT_INSTALMENT'] + 10,'PAYMENT_MISS_FLAG'] = 1
installments_agg_dict = {
        'PAYMENT_MISS_FLAG': ['max']
    }

installments_agg = installments_payments_df.groupby('SK_ID_CURR').agg({**installments_agg_dict})
installments_agg.columns = pd.Index(['INST_' + e[0] + "_" + e[1].upper() for e in installments_agg.columns.tolist()])

In [0]:

credit_balance_agg_dict = {
'MONTHS_BALANCE':['max'],
'AMT_BALANCE':['max'],
'AMT_CREDIT_LIMIT_ACTUAL':['max'],
'AMT_DRAWINGS_ATM_CURRENT':['max'],
'AMT_DRAWINGS_CURRENT':['max'],
'AMT_DRAWINGS_OTHER_CURRENT':['max'],
'AMT_DRAWINGS_POS_CURRENT':['max'],
'AMT_INST_MIN_REGULARITY':['min'],
'AMT_PAYMENT_CURRENT':['max'],
'AMT_PAYMENT_TOTAL_CURRENT':['max'],
'AMT_RECEIVABLE_PRINCIPAL':['max'],
'AMT_RECIVABLE':['max'],
'AMT_TOTAL_RECEIVABLE':['max'],
'CNT_DRAWINGS_ATM_CURRENT':['max'],
'CNT_DRAWINGS_CURRENT':['max'],
'CNT_DRAWINGS_OTHER_CURRENT':['max'],
'CNT_DRAWINGS_POS_CURRENT':['max'],
'CNT_INSTALMENT_MATURE_CUM':['max'],
'SK_DPD':['max'],
'SK_DPD_DEF':['max']
}
credit_balance_agg = credit_balance_df.groupby('SK_ID_CURR').agg({**credit_balance_agg_dict})
credit_balance_agg.columns = pd.Index(['CRED_' + e[0] + "_" + e[1].upper() for e in credit_balance_agg.columns.tolist()])

In [0]:
# POS_CASH_balance
POS_agg_dict = {
        'SK_DPD_DEF': ['max']
    }

POS_agg = POS_CASH_balance_df.groupby('SK_ID_CURR').agg({**POS_agg_dict})
POS_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in POS_agg.columns.tolist()])

In [0]:
# Previous Application
prev_app_agg_dict = {
        'AMT_CREDIT': ['min']
    }

prev_app_agg = previous_application_df.groupby('SK_ID_CURR').agg({**prev_app_agg_dict})
prev_app_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_app_agg.columns.tolist()])

In [0]:
# final merged dataset
train = application_train_df.merge(bureau_agg,how='left',on='SK_ID_CURR')\
                            .merge(installments_agg,how='left',on='SK_ID_CURR')\
                            .merge(POS_agg,how='left',on='SK_ID_CURR')\
                            .merge(credit_balance_agg, how='left',on='SK_ID_CURR')

test = application_test_df.merge(bureau_agg,how='left',on='SK_ID_CURR')\
                            .merge(installments_agg,how='left',on='SK_ID_CURR')\
                            .merge(POS_agg,how='left',on='SK_ID_CURR')\
                            .merge(credit_balance_agg, how='left',on='SK_ID_CURR')

In [0]:
#one-hot encoding
#input: features (no label), test_features

#remove the label before encoding
labels = train['TARGET'] 
features = train.drop(columns = ['TARGET'])
features = pd.get_dummies(features)
test_features = pd.get_dummies(test)
#test_features.columns
# Align the dataframes by the columns
features, test_features = features.align(test_features, join = 'inner', axis = 1)

In [0]:
### Remove NA

# Checking missing percentage in different columns
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
#pd.set_option('display.max_rows', 300)
#missing_data(features)
# Number of variables with missing values
#len(missing_data(features).Percent[missing_data(features).Percent>0].index)

treatment_list = list(missing_data(features).Percent[missing_data(features).Percent>0].index)
# The reason for them not being populated is whether the account is closed, or that field does not pertain for the customer on record.
# For eg. 
# If a customer closes the account, his days past due, balance etc. information will not be populated in the system
# and hence missing values
# so that is as good as 0.
# same is for other variables

# For fields like owned house area, where they are missing...
# if the user has no owned house...
# the associated fields will be blank..
# that is also as good as 0 area, 0 rooms


# Replacing the features with zero

features[treatment_list] = features[treatment_list].replace({np.nan:'0'})

#do the same thing for test data
test_treatment_list = list(missing_data(test_features).Percent[missing_data(test_features).Percent>0].index)
test_features[test_treatment_list] = test_features[test_treatment_list].replace({np.nan:'0'})
# Ensuring if replacement worked - should have zero missing values now

#missing_data(test_features)

In [0]:
### scaling

import numpy as np
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(features), columns=features.columns,index = features.index)
test_scaled = pd.DataFrame(scaler.transform(test_features), columns=test_features.columns)
train_scaled2 = pd.concat([labels,train_scaled],axis = 1)

In [0]:
#missing_data(features)

Features selection

[link to features selection](https://www.kaggle.com/sz8416/6-ways-for-feature-selection)


In [0]:
#Pearson-correlation selector
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

def cor_selector(X, y):
    feature_name = X.columns.tolist()
    cor_list = []
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-120:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

cor_support, cor_feature = cor_selector(train_scaled2.loc[:,train_scaled2.columns != 'TARGET'], train_scaled2['TARGET'])
print(str(len(cor_feature)), 'selected features')


120 selected features


In [0]:
X_fs = train_scaled2.loc[:,train_scaled2.columns != 'TARGET']
y_fs = train_scaled2['TARGET']

In [0]:
#chi-square selector
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

X_fsnorm = MinMaxScaler().fit_transform(X_fs)
chi_selector = SelectKBest(chi2, k=120)
chi_selector.fit(X_fsnorm, y_fs)

chi_support = chi_selector.get_support()
chi_feature = X_fs.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

120 selected features


In [0]:
#random forest selector

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier


embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), threshold='1.25*median')
embeded_rf_selector.fit(X_fs, y_fs)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X_fs.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

128 selected features


In [0]:
### keep columns that are selected by all selector

pd.set_option('display.max_rows', None)
# put all selection together
feature_name = X_fs.columns.tolist()
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 
                                     'Chi-2':chi_support,
                                    'Random Forest':embeded_rf_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(130)

,Feature,Pearson,Chi-2,Random Forest,Total
1,YEARS_BUILD_MODE,True,True,True,3
2,YEARS_BUILD_MEDI,True,True,True,3
3,YEARS_BUILD_AVG,True,True,True,3
4,YEARS_BEGINEXPLUATATION_MODE,True,True,True,3
5,YEARS_BEGINEXPLUATATION_MEDI,True,True,True,3
6,YEARS_BEGINEXPLUATATION_AVG,True,True,True,3
7,TOTALAREA_MODE,True,True,True,3
8,REG_CITY_NOT_WORK_CITY,True,True,True,3
9,REG_CITY_NOT_LIVE_CITY,True,True,True,3
10,REGION_RATING_CLIENT_W_CITY,True,True,True,3


In [0]:
 
#colnames of features that met the standard of all three selectors.
selected_features_list = feature_selection_df[feature_selection_df.Total >= 1]['Feature'].tolist()

Modeling

In [0]:
#features.iloc[:10,:][selected_features_list]

### setup training data and test data
y_train = labels
X_train = features[selected_features_list]
X_test = test_features[selected_features_list]

In [0]:
X_test.to_csv('x_test_data.csv')

Baseline logistic

In [0]:
X_train.to_csv('x_train_data_3.csv')
y_train.to_csv('y_train_data_3.csv')
X_test.to_csv('x_test_data_3.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [0]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
scoring = ['precision_macro', 'recall_macro']
logistic = LogisticRegression()
scores = cross_validate(logistic, X_train, y_train, scoring=scoring,cv=5)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [0]:
scores['test_precision_macro']

array([0.55964096, 0.62631445, 0.79298308, 0.53106757, 0.45963283])

NN

In [0]:
from sklearn.model_selection import train_test_split
from keras.models import Model,Sequential
from keras import optimizers

from keras.layers import GlobalMaxPooling2D,Dense,Dropout

model = Sequential()
model.add(Dense(260, input_shape = (262,), activation='relu'))


model.add(Dense(200, activation='relu', kernel_initializer='random_normal'))

model.add(Dense(150, activation='relu', kernel_initializer='random_normal'))

model.add(Dense(100, activation='relu', kernel_initializer='random_normal'))

model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

model.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

model.fit(train_2,y_train, batch_size=10, epochs=100)